In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import csv


In [3]:
import csv
# 파일 읽어오기
f = open('../Losangeles_url_list.csv', 'r')
rdr = csv.reader(f)
co =  1
# 텍스트에 있는 URL list에 첨가하는 작업
URLLIST = list()
for line in rdr:
    for i in line:
        co = co +1
        URLLIST.append(i)
print(co)
f.close()

104


# 가게 시간정보 모델링 데이터

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
options.add_argument('disable-gpu')    # GPU 사용 안함
options.add_argument('lang=ko_KR')
dayday = list()
open_day = dict()
for URL in URLLIST[:10]:
    restaurant = dict()
    driver = webdriver.Chrome('../chromedriver', options=options)
    driver.get(URL)
    div_e1gr019l6 =driver.find_element_by_class_name('e1gr019l6')

    title = URL.split('/')[-1]
    restaurant['url'] = URL
    
    
    ################################배달 시간
    try:
        what_delivery=div_e1gr019l6.find_element_by_tag_name('h3').get_attribute('innerHTML').replace('<span>','').replace('</span>','')
        restaurant[what_delivery] = div_e1gr019l6.find_element_by_tag_name('div').get_attribute('innerHTML').replace('<span>','').replace('</span>','')
    except:
        print('배달시간 데이터 없는건가?')
    #################################배달 시간


    
    open_hour = dict()
    #################################식당 오픈시간
    div_efoongq0=driver.find_elements_by_class_name('efoongq0')

    for i in div_efoongq0:
        for a in i.find_elements_by_tag_name('div'):
            try:
                if int == type(int(a.get_attribute('innerHTML').replace('<span>','').replace('</span>','')[0])):
                    open_hour[what_hour] =a.get_attribute('innerHTML').replace('<span>','').replace('</span>','')

            except:
                what_hour = a.get_attribute('innerHTML').replace('<span>','').replace('</span>','')
            
        dayday.append(restaurant)
    restaurant['hour'] = open_hour
    ##################################
    open_day[title] = restaurant

In [5]:
open_day

{'jack-in-the-box-los-angeles-516-n-beaudry-ave': {'url': 'https://postmates.com/merchant/jack-in-the-box-los-angeles-516-n-beaudry-ave',
  'Open Hours': '',
  'hour': {'Open 24 hours': '5:00 AM-4:30 AM'}},
 'popeyes-louisiana-kitchen-los-angeles': {'url': 'https://postmates.com/merchant/popeyes-louisiana-kitchen-los-angeles',
  'Open Hours': 'Order Delivery until 4:00 AM',
  'hour': {'Monday': '5:00 AM-4:30 AM',
   'Tuesday': '5:00 AM-4:30 AM',
   'Wednesday': '5:00 AM-4:30 AM',
   'Thursday': '5:00 AM-4:30 AM',
   'Friday': '5:00 AM-4:30 AM',
   'Saturday': '5:00 AM-4:30 AM',
   'Sunday': '5:00 AM-4:30 AM'}},
 'dennys-530-ramirez-st': {'url': 'https://postmates.com/merchant/dennys-530-ramirez-st',
  'Open Hours': 'Order Delivery until 4:00 AM',
  'hour': {'Monday': '5:00 AM-4:30 AM',
   'Tuesday': '5:00 AM-4:30 AM',
   'Wednesday': '5:00 AM-4:30 AM',
   'Thursday': '5:00 AM-4:30 AM',
   'Friday': '5:00 AM-4:30 AM',
   'Saturday': '5:00 AM-4:30 AM',
   'Sunday': '5:00 AM-4:30 AM'}},
 

# store 모델 데이터


In [6]:
# open_day

# 여기에 다 있습니다!!!

In [7]:

for restaurant_data in open_day:
#     restaurant_name = restaurant_data
    URL = open_day[restaurant_data]['url']
    print(URL)
    res = requests.post(URL + '?client=customer.web&version=3.0.0')
    data=  res.json()

    open_day[restaurant_data]['name'] = data['data']['place']['name']
    try:
        open_day[restaurant_data]['description'] = data['data']['place']['brand']['description']
    except KeyError:
        open_day[restaurant_data]['description'] =''
#         print('설명이 없거나 brand가 없거나')
    

    ########################is_delivery 계산해야함 여기서!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    open_day[restaurant_data]['is_delivery'] = data['data']['place']['hours']['state']

    ######################## is_pickup
    try:
        open_day[restaurant_data]['is_pickup'] = data['data']['place']['is_pickup_enabled']
    except KeyError:
        open_day[restaurant_data]['is_pickup'] = False
#         print('배달이 안되는 건가 빈건가?')
    ########################delivery_price
    try:
        open_day[restaurant_data]['delivery_fee'] = float(data['data']['place']['delivery_fee_badge'].split(' ')[0][1::])
    except:
        open_day[restaurant_data]['delivery_fee'] = 0
#         print('Free Delivery  $0')

    ########################estimated_prep_time(만드는데 걸리는 시간)
    open_day[restaurant_data]['estimated_prep_time'] = data['data']['place']['estimated_prep_time']

    #########################
    try:
        open_day[restaurant_data]['is_partner'] =data['data']['place']['is_partner']
    except KeyError:
        open_day[restaurant_data]['is_partner'] = False
#         print('파트너가 아니구나')

    ########################thumbnail image(둘중 하나 쓰기)
    open_day[restaurant_data]['store_img'] = data['data']['place']['header_img']['original_url']
#     print(data['data']['place']['header_img']['resolutions'][0]['url'])

    ########################state?????

    ########################address
    open_day[restaurant_data]['street_address_1'] = data['data']['place']['street_address_1']
    open_day[restaurant_data]['city'] = data['data']['place']['city']


    ######################## lat(위도)
    open_day[restaurant_data]['lat'] = data['data']['place']['lat']

    ######################## lng(경도)
    open_day[restaurant_data]['lng'] = data['data']['place']['lng']
    
    ######################## lng(경도)
    try:
        open_day[restaurant_data]['badges'] = data['data']['place']['badges']
    except:
        open_day[restaurant_data]['badges'] = ''
        

    ########################badge
    # selenium !!!! selenium !!!! selenium !!!! 

    ########################food type
    open_day[restaurant_data]['food_type'] = data['data']['place']['primary_place_category']['slug']
    
    food_item = dict()
    
    try:
        for category in data['data']['place']['catalog']['categories']:

            
            categori_food_list = dict()
            
            for product in category['products']:
                item_info = dict()
                item_info['name']  = product['name']
                
                try:
                    item_info['img_url'] = product['img']['resolutions'][0]['url']
                except KeyError:
#                     print('이미지가 없다')
                    item_info['img_url'] = ''
                
                try:
                    item_info['description'] = product['description']
                except KeyError:
#                     print('설명이 없는거다')            
                    item_info['description'] = ''
                
                item_info['base_price'] = product['base_price']
#                 print('item_info==============' ,item_info )
                categori_food_list[item_info['name']] = item_info
    
    
#             print('categori_food_list=============================>', categori_food_list )
                
#########################################################################################
                food_options = dict()
                options_list = list()
                try:
                    for options in product['option_groups']:
                        option_name = item_info['name']+ ' = option_name = '  +options['name'] 

                        
                        #################################
                        option_category = dict()
                        option_list = list()
                        for option in options['options']:
                            option_info = dict()

                            option_info['name'] = option['name']

                            
                            
                            option_info['option_price'] = option['price']
#                             option_info['select'] = False
                            option_list.append(option_info)
#                         print(option_list)
                        option_category[options['name']] = option_list
                        try:
                            option_category['is_required'] = options['single']
                        except:
                            option_category['is_required'] = False
#                         print('option_category \n \n',option_category)
#                         print('\n \n')
                        options_list.append(option_category)
#                     print(options_list)
                    item_info['options'] = options_list
#                     print(item_info)
                except:
                    print('옵션이 없습니다!!!')
                    item_info['options'] = ''

                categori_food_list[item_info['name']] = item_info
            food_item[category['name']] = categori_food_list
    except:
        print('카탈로그가 비어있는거 이거는 음식정보가 없어서 제거해야 하는데')
        food_item['categori'] =''
        continue
    open_day[restaurant_data]['menu'] = food_item
    



https://postmates.com/merchant/jack-in-the-box-los-angeles-516-n-beaudry-ave
https://postmates.com/merchant/popeyes-louisiana-kitchen-los-angeles
https://postmates.com/merchant/dennys-530-ramirez-st
https://postmates.com/merchant/taco-bell-103-s-rampart-blvd
https://postmates.com/merchant/farmer-boys-los-angeles-3
https://postmates.com/merchant/subway-los-angeles-7
https://postmates.com/merchant/ab3ed64c-dc98-486f-b95a-ff884506d43d
https://postmates.com/merchant/alessandros-place-los-angeles
https://postmates.com/merchant/winchells-los-angeles-2501-s-san-pedro-st
https://postmates.com/merchant/5df3eb19-a8af-4bc5-8628-c6f41d59380e


# 각 가게마다 카테고리별 메뉴들 (item)

In [8]:
open_day

{'jack-in-the-box-los-angeles-516-n-beaudry-ave': {'url': 'https://postmates.com/merchant/jack-in-the-box-los-angeles-516-n-beaudry-ave',
  'Open Hours': '',
  'hour': {'Open 24 hours': '5:00 AM-4:30 AM'},
  'name': 'Jack in the Box',
  'description': "Burgers, fries, tacos, breakfast, pure culinary bliss—is there anything you can't get at Jack in the Box? With delicious menu options any time of day, this chain does the classics right while thinking outside of the box for new treats to try. From inventive breakfast offerings to trendy shake flavors, you're sure to find something to tickle your tastebuds at Jack in the Box. No time for a trip to pick up your square meal? Let Postmates bring it to you, so you can focus on savoring every single bite. Hop online or in our app to get your Jack in the Box favorites ASAP.",
  'is_delivery': 'open',
  'is_pickup': False,
  'delivery_fee': 2.99,
  'estimated_prep_time': 0,
  'is_partner': False,
  'store_img': 'com.postmates.img.prod.s3.amazona

In [11]:
from stores.models import *

In [12]:
Store.objects.create(name)

stores.models.Store

In [20]:
for key in open_day.keys():
#     print(open_day[key])
    Store.objects.create(
        name = open_day[key]['name'],
        description = open_day[key]['description']
        pickup = open_day[key]['is_pickup']
        delivery_price = open_day[key]['is_delivery']
        estimated_prep_time = open_day[key]['estimated_prep_time']
        is_partner = open_day[key]['is_partner']
        thumb
    
    )
    print(open_day[key]['name'])
    print(open_day[key]['description'])
    print(open_day[key]['is_pickup'])
    print(open_day[key]['is_delivery'])
    print(open_day[key]['delivery_fee'])
    print(open_day[key]['estimated_prep_time'])
    print(open_day[key]['is_partner'])
    print(open_day[key]['store_img'])
    print(open_day[key]['street_address_1'])
    print(open_day[key]['lat'])
    print(open_day[key]['lng'])
    print(open_day[key]['badges'])
    print(open_day[key]['food_type'])

Jack in the Box
Burgers, fries, tacos, breakfast, pure culinary bliss—is there anything you can't get at Jack in the Box? With delicious menu options any time of day, this chain does the classics right while thinking outside of the box for new treats to try. From inventive breakfast offerings to trendy shake flavors, you're sure to find something to tickle your tastebuds at Jack in the Box. No time for a trip to pick up your square meal? Let Postmates bring it to you, so you can focus on savoring every single bite. Hop online or in our app to get your Jack in the Box favorites ASAP.
False
open
2.99
0
False
com.postmates.img.prod.s3.amazonaws.com/c590876c-cacc-4da8-aab1-ff3349f51d67/orig.jpg
516 N Beaudry Ave
34.0647612
-118.2492018

fast-food
Popeyes Louisiana Kitchen
Craving some good, old fashioned Louisiana-style chicken? It's hard to beat Popeye's. Since it first hatched out of a delicious fried, er, egg in 1972 New Orleans, this fast favorite has been serving up fried chicken in a

In [ ]:
print(open_day[key]['delivery_fee'])

In [81]:
food_item = dict()

for URL in URLLIST:
    print(URL)
    res = requests.post(URL + '?client=customer.web&version=3.0.0')
    data=  res.json()
    try:
        for category in data['data']['place']['catalog']['categories']:

            item_info = dict()

            for product in category['products']:
                item_info['name']  = product['name']
                print(product['name'])
                try:
                    print(product['img']['resolutions'][0]['url'])
                    item_info['img_url'] = product['img']['resolutions'][0]['url']
                except KeyError:
                    print('이미지가 없다')
                    item_info['img_url'] = ''
                try:
                    item_info['description'] = product['description']
                except KeyError:
                    print('설명이 없는거다')            
                    item_info['description'] = ''
                item_info['base_price'] = product['base_price']
            print('-----------------------------')
            food_item[category['name']] = item_info
    except:
        print('카탈로그가 비어있는거 이거는 음식정보가 없어서 제거해야 하는데')
        food_item['categori'] =''
        continue


https://postmates.com/merchant/jack-in-the-box-los-angeles-516-n-beaudry-ave
Tiny Tacos
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Fbfacadf9-600d-4edb-b32f-067b936fe5ea%2Foriginal.jpg%3Fv%3D63760427548&width=1280
Loaded Tiny Tacos
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Ff85fd2f7-ac15-4b72-a00f-921aa273b9ad%2Foriginal.jpg%3Fv%3D63760427576&width=1280
Classic Popcorn Chicken Combo (Large)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F4a043bb6-3527-47a7-af6a-371069b03679%2Foriginal.jpg%3Fv%3D63760427627&width=1280
Spicy Popcorn Chicken Combo (Large)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F11af102a-126c-411f-961f-01f994ea739d%2Foriginal.jpg%3Fv%3D63760427634&width=1280
Southwest Cheddar Cheeseburger A La Carte
ht

Chicken Combo (3 Pcs)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Feda8fb22-b7bb-45ac-8793-a61282f2047b%2Foriginal.jpg%3Fv%3D63722056428&width=1280
-----------------------------
Chicken Dinner (2 Pcs)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F5c539750-6cbe-48bf-be8c-5440edc18ca4%2Foriginal.jpg%3Fv%3D63722056056&width=1280
Chicken Dinner (3 Pcs)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Fa289022a-ce41-4d5b-84ee-78287735946c%2Foriginal.jpg%3Fv%3D63722056067&width=1280
-----------------------------
Handcrafted Tenders (8 Pcs)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F23ab5399-8adf-4c22-a3eb-b37d8369108e%2Foriginal.jpg%3Fv%3D63722056227&width=1280
Handcrafted Tenders (12 Pcs)
https://raster-static.postmates.com?hei

카탈로그가 비어있는거 이거는 없애야 하는데
https://postmates.com/merchant/farmer-boys-los-angeles-3
Breakfast Burrito
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F4a5c1828-b3e3-4046-99de-e11fc7a5831e%2Foriginal.jpg%3Fv%3D63701795464&width=1280
French Toast Platter
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F11ea157e-c23e-4b84-bdcf-9c0893621eee%2Foriginal.jpg%3Fv%3D63701795867&width=1280
3 French Toast Slices
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Fa399d585-e945-4f0d-97df-1d45891033c3%2Foriginal.jpg%3Fv%3D63701795881&width=1280
Hot Cakes Platter
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F890e522d-34a6-422c-8c7c-7a7a81c9ed7f%2Foriginal.jpg%3Fv%3D63701795913&width=1280
4 Hot Cakes
https://raster-static.postmates.com?height=1280&url=

Oven Roasted Chicken  6 Inch Regular Sub
https://raster-static.postmates.com?height=640&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F9193f04d-846d-5f82-886e-b2c62345cbe4%2F9193f04d-846d-5f82-886e-b2c62345cbe4.png%3Fv%3D63759209635&width=640
Oven Roasted Chicken  Footlong Regular Sub
https://raster-static.postmates.com?height=640&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F9193f04d-846d-5f82-886e-b2c62345cbe4%2F9193f04d-846d-5f82-886e-b2c62345cbe4.png%3Fv%3D63759209635&width=640
Veggie Delite® 6 Inch Regular Sub
https://raster-static.postmates.com?height=640&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Fd111011d-3aeb-53a0-9008-174d15ecc6c0%2Fd111011d-3aeb-53a0-9008-174d15ecc6c0.png%3Fv%3D63759510009&width=640
Veggie Delite® Footlong Regular Sub
https://raster-static.postmates.com?height=640&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Fd111011d-3aeb-53a0-9008-174d15ecc6c0%2Fd111011d-3aeb-53a0-9008-174d15ecc6c

Hamburger
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F0ab01c63-e634-4998-914b-03335735f676%2Foriginal.jpg%3Fv%3D63753252796&width=1280
Hamburger (1/4 lb.) 
이미지가 없다
Cheeseburger
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F7147fc89-4cdb-4232-8535-7bba5a097e46%2Foriginal.jpg%3Fv%3D63753252855&width=1280
Cheeseburger (1/4 lb.)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F4e0e2f98-7852-4f83-a573-2bedf2b6c389%2Foriginal.jpg%3Fv%3D63753252928&width=1280
Double Hamburger
이미지가 없다
Double Hamburger (1/4 lb.)
이미지가 없다
Double Cheeseburger
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Fd594ebd3-d1bf-4183-b3f6-7d8ff554b631%2Foriginal.jpg%3Fv%3D63753253126&width=1280
Double Cheeseburger (1/4 lb.)
이미지가 없다
Caliente Burger
https://raster-

...

Egg Roll
이미지가 없다
Cold Jelly Fish
이미지가 없다
Cold Jelly Fish with Walnut & Preserved Egg
이미지가 없다
설명이 없는거다
Sliced BBQ Pork
이미지가 없다
설명이 없는거다
Deep Fried Prawns
이미지가 없다
설명이 없는거다
Deep Fried Pork Intestine
이미지가 없다
설명이 없는거다
Pan Fried Dumplings
이미지가 없다
설명이 없는거다
Mini Pork Buns
이미지가 없다
-----------------------------
Crab Meat with Fish Maw Soup
이미지가 없다
설명이 없는거다
Fresh Fish with Bean Curd Soup
이미지가 없다
설명이 없는거다
Assorted Seafood Soup
이미지가 없다
설명이 없는거다
Shredded Duck with Meat Soup
이미지가 없다
설명이 없는거다
West Lake Style Beef Soup
이미지가 없다
설명이 없는거다
Chicken Cream Corn Soup
이미지가 없다
설명이 없는거다
Hot & Sour Soup
이미지가 없다
Sizzling Rice Soup
이미지가 없다
설명이 없는거다
Abalone with Sliced Chicken Soup
이미지가 없다
설명이 없는거다
Assorted Meat with Diced Winter Melon Soup
이미지가 없다
설명이 없는거다
-----------------------------
Bird's Nest with Crab Meat Soup
이미지가 없다
설명이 없는거다
Bird's Nest with Shredded Chicken Soup
이미지가 없다
설명이 없는거다
-----------------------------
Wor Wonton Soup
이미지가 없다
설명이 없는거다
Chicken Wonton Soup
이미지가 없다
설명이 없는거다
Shrimp Wonton Soup
이미지가 없다
설명

Custom Organic Stir-Fry!
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Fc327823c-d670-4c60-bb67-088cee249e4d%2Foriginal.jpg%3Fv%3D63719734308&width=1280
Custom Organic Bowl or Salad
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F5df23b9b-aca1-4eb6-a9ca-11db93d5527b%2Foriginal.jpg%3Fv%3D63719734324&width=1280
Custom Organic Tacos!
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F3245c295-2c6c-4335-9f4d-d187b35f6db1%2Foriginal.jpg%3Fv%3D63719734330&width=1280
Custom Organic Quesadilla!
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F5b2a386d-f55c-4f28-ae1f-1c4a245a4f4b%2Foriginal.jpg%3Fv%3D63719734300&width=1280
Custom Organic Wrap
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fupload

Gamjatang (Pork Bone Soup/감자탕)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F885c7778-fee4-4f94-906f-012fe5822f86%2Foriginal.jpg%3Fv%3D63742629426&width=1280
Yukgaejang (Spicy Beef Soup/육개장)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F6f0f5898-fa26-4ab2-9caa-dfa0e4010864%2Foriginal.jpg%3Fv%3D63742629475&width=1280
Galbitang (Beef Short Rib Soup/갈비탕)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F57423e16-0586-4f7c-9971-adc0e4bd7edf%2Foriginal.jpg%3Fv%3D63742629417&width=1280
Kimchi Jjigae (Spicy Kimchi Stew/김치찌개)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F44cf8550-a522-46cd-9b66-145b9ac2e537%2Foriginal.jpg%3Fv%3D63742629437&width=1280
Soondubu Jjigae (Soft Tofu Stew/순두부찌개)
https://raster-static.postmates.com?height=128

Shoyu Tonkotsu Ramen (Classic)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F473e2f38-f53d-481f-9fca-ec56e368ad18%2Foriginal.jpeg%3Fv%3D63754412850&width=1280
Shio Tonkotsu Ramen (White)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F340f8f11-1173-4c58-b99f-e1d72d8ace10%2Foriginal.jpeg%3Fv%3D63754412864&width=1280
Ma! Kuro Tonkotsu Ramen (Black)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F62dc6d7e-b423-4418-a44a-44463278dce3%2Foriginal.jpeg%3Fv%3D63754412876&width=1280
Akai Tonkotsu Ramen (Red)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F0c07ed8d-30c4-45d8-9e6a-77065af9b435%2Foriginal.jpeg%3Fv%3D63754412890&width=1280
Miso Tonkotsu Ramen
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.post

Spinach Dip
이미지가 없다
Crudites
이미지가 없다
Tomato Soup
이미지가 없다
설명이 없는거다
Cup of Chili
이미지가 없다
설명이 없는거다
Chili-Cheese Fries & Tots
이미지가 없다
-----------------------------
Caesar Salad
이미지가 없다
Iceberg Wedge Salad
이미지가 없다
Seasonal Salad
이미지가 없다
-----------------------------
Turkey Salad Sandwich
이미지가 없다
설명이 없는거다
Tuna Salad Sandwich
이미지가 없다
설명이 없는거다
Grilled Cheese Sandwich & Tomato Soup
이미지가 없다
설명이 없는거다
Bowl of Chili
이미지가 없다
설명이 없는거다
Big Mac ‘N’ Cheese
이미지가 없다
설명이 없는거다
-----------------------------
Bourbon Pecan Pie
이미지가 없다
설명이 없는거다
Chocolate Cream Pie
이미지가 없다
설명이 없는거다
Banana Cream Pie
이미지가 없다
설명이 없는거다
Seasonal Pie
이미지가 없다
Churro Waffle A La Mode
이미지가 없다
설명이 없는거다
Rootbeer & Coke Float
이미지가 없다
설명이 없는거다
-----------------------------
카탈로그가 비어있는거 이거는 없애야 하는데
https://postmates.com/merchant/las-perlas-los-angeles
Carnitas Tacos
이미지가 없다
Pollo Tacos
이미지가 없다
Carne Asada Tacos
이미지가 없다
-----------------------------
Potato Taquitos
이미지가 없다
Pollo Taquitos
이미지가 없다
-----------------------------
Fresh Chips & Salsa

In [111]:
open_day

{'jack-in-the-box-los-angeles-516-n-beaudry-ave': {'url': 'https://postmates.com/merchant/jack-in-the-box-los-angeles-516-n-beaudry-ave',
  'Open Hours': '',
  'hour': {},
  'name': 'Jack in the Box',
  'description': "Burgers, fries, tacos, breakfast, pure culinary bliss—is there anything you can't get at Jack in the Box? With delicious menu options any time of day, this chain does the classics right while thinking outside of the box for new treats to try. From inventive breakfast offerings to trendy shake flavors, you're sure to find something to tickle your tastebuds at Jack in the Box. No time for a trip to pick up your square meal? Let Postmates bring it to you, so you can focus on savoring every single bite. Hop online or in our app to get your Jack in the Box favorites ASAP.",
  'is_delivery': 'open',
  'delivery_fee_badge': 2.99,
  'estimated_prep_time': 0,
  'original_url': 'com.postmates.img.prod.s3.amazonaws.com/c590876c-cacc-4da8-aab1-ff3349f51d67/orig.jpg',
  'street_addre

In [15]:
f = open('san_fransisco.json' ,'w')
f.write(str(open_day))
f.close()

In [22]:
import json
json_str = json.dumps(open_day)
data = json.loads(json_str)

In [24]:
data

{'jack-in-the-box-los-angeles-516-n-beaudry-ave': {'url': 'https://postmates.com/merchant/jack-in-the-box-los-angeles-516-n-beaudry-ave',
  'Open Hours': '',
  'hour': {},
  'name': 'Jack in the Box',
  'description': "Burgers, fries, tacos, breakfast, pure culinary bliss—is there anything you can't get at Jack in the Box? With delicious menu options any time of day, this chain does the classics right while thinking outside of the box for new treats to try. From inventive breakfast offerings to trendy shake flavors, you're sure to find something to tickle your tastebuds at Jack in the Box. No time for a trip to pick up your square meal? Let Postmates bring it to you, so you can focus on savoring every single bite. Hop online or in our app to get your Jack in the Box favorites ASAP.",
  'is_delivery': 'open',
  'is_pickup': False,
  'delivery_fee_badge': 2.99,
  'estimated_prep_time': 0,
  'is_partner': False,
  'original_url': 'com.postmates.img.prod.s3.amazonaws.com/c590876c-cacc-4da8

In [80]:
res = requests.post('https://postmates.com/merchant/jack-in-the-box-los-angeles-516-n-beaudry-ave' + '?client=customer.web&version=3.0.0')
data=  res.json()
for category in data['data']['place']['catalog']['categories']:
#     print(category)
        for product in category['products']:
            print(product['name'])
            print(product['img']['resolutions'][0]['url'])


Tiny Tacos
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Fbfacadf9-600d-4edb-b32f-067b936fe5ea%2Foriginal.jpg%3Fv%3D63760427548&width=1280
Loaded Tiny Tacos
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2Ff85fd2f7-ac15-4b72-a00f-921aa273b9ad%2Foriginal.jpg%3Fv%3D63760427576&width=1280
Classic Popcorn Chicken Combo (Large)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F4a043bb6-3527-47a7-af6a-371069b03679%2Foriginal.jpg%3Fv%3D63760427627&width=1280
Spicy Popcorn Chicken Combo (Large)
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.postmates.com%2Fuploads%2Fmedia%2F11af102a-126c-411f-961f-01f994ea739d%2Foriginal.jpg%3Fv%3D63760427634&width=1280
Southwest Cheddar Cheeseburger A La Carte
https://raster-static.postmates.com?height=1280&url=https%3A%2F%2Fitems-static.

# 각 음심마다 토핑 옵션

In [ ]:
for food in data['data']['place']['catalog']['categories']:
#     print('food_type' , food['name'])
    
    for product in food['products']:
#         print('    food_name' , product['name'])
        
        for options in product['option_groups']:
            ###########Topping category
            print('          options_category   : ' ,options['name'])
            
            for option in options['options']:
                ############topping name & topping price
                print('                           ' ,option['name'] , '   :  ',option['price'])

        print('-----------------------------')
